In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
%cd /content/drive/MyDrive/proyectos3/
!ls

/content/drive/MyDrive/proyectos3
alexNet.ipynb  efficientNet.ipynb  models    PlantVillage_bgremoved  wandb
cnn.ipynb      GoogleNet.ipynb	   PlantDoc  resNet.ipynb


# Cargar Conjunto de Datos

In [39]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import ConcatDataset


train_dir0 = './PlantVillage_bgremoved/train'
train_dir1 = './PlantDoc/train/'

val_dir0 = './PlantVillage_bgremoved/valid'
val_dir1 = './PlantDoc/test/'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset0 = ImageFolder(train_dir0, transform=transform,  is_valid_file=lambda x: x.endswith('.JPG'))
train_dataset1 = ImageFolder(train_dir1, transform=transform)

val_dataset0 = ImageFolder(val_dir0, transform=transform, is_valid_file=lambda x: x.endswith('.JPG'))
val_dataset1 = ImageFolder(val_dir1, transform=transform)

train_dataset = ConcatDataset([train_dataset0, train_dataset1])
val_dataset = ConcatDataset([val_dataset0, val_dataset1])

classes = train_dataset0.classes

classes

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

crear carga de datos para facilitar el acceso a los datos

In [40]:
from torch.utils.data import DataLoader

batch_size = 32
num_workers = 8 # Número de procesos que se utilizarán para cargar datos en paralelo
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [41]:
import torch

def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
        return len(self.dl)

In [42]:
device = get_default_device()

train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)

In [43]:
device

device(type='cuda')

In [44]:
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import precision_score, f1_score

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


class ImageClassificationBase(nn.Module):

    def training_step(self, batch):
        "calculate loss for a batch of training data"
        images, labels = batch
        out = self(images)                  # Generate predictions
        # nn.CrossEntropyLoss combines nn.LogSoftmax and nn.NLLLoss
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss

    def validation_step(self, batch):
        "calculate loss, accuracy, precision and f1 score for a batch of validation data"
        images, labels = batch
        out = self(images)                   # Generate prediction
        # nn.CrossEntropyLoss combines nn.LogSoftmax and nn.NLLLoss
        loss = F.cross_entropy(out, labels)  # Calculate loss
        preds = torch.argmax(out, dim=1)
        acc = accuracy(out, labels)          # Calculate accuracy

        # calculate precision and f1 score
        precision = precision_score(labels.cpu(), preds.cpu(), average='weighted', zero_division=1)
        f1 = f1_score(labels.cpu(), preds.cpu(), average='macro')
        return {'val_loss': loss.detach(), 'val_accuracy': acc, 'val_precision': precision, 'val_f1': f1}

    def validation_epoch_end(self, outputs):
        batch_losses = [x["val_loss"] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()       # Combine loss
        batch_accuracy = [x["val_accuracy"] for x in outputs]
        epoch_accuracy = torch.stack(batch_accuracy).mean()
        return {"val_loss": epoch_loss.item(), "val_accuracy": epoch_accuracy.item()} # Combine accuracies

    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_accuracy']))


In [45]:
# CNN
def ConvBlock(in_channels, out_channels, pool=False):
    layers = [
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
        ]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class CNN(ImageClassificationBase):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = ConvBlock(3, 32)
        self.conv2 = ConvBlock(32, 64, pool=True)

        self.conv3 = ConvBlock(64, 128)
        self.conv4 = ConvBlock(128, 128, pool=True)

        self.conv5 = ConvBlock(128, 256)
        self.conv6 = ConvBlock(256, 256, pool=True)

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), # output size = 1x1
            nn.Flatten(),
            nn.Linear(256, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes)
        )

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        out = self.conv6(out)
        out = self.classifier(out)
        return out

In [46]:
model = CNN(len(classes))
model = to_device(model, device)

In [47]:
from torchsummary import summary              # for getting the summary of our model

# getting summary of the model
INPUT_SHAPE = (3, 224, 224)
print(summary(model.cuda(), (INPUT_SHAPE)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
       BatchNorm2d-2         [-1, 32, 224, 224]              64
              ReLU-3         [-1, 32, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          18,496
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
         MaxPool2d-7         [-1, 64, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]          73,856
       BatchNorm2d-9        [-1, 128, 112, 112]             256
             ReLU-10        [-1, 128, 112, 112]               0
           Conv2d-11        [-1, 128, 112, 112]         147,584
      BatchNorm2d-12        [-1, 128, 112, 112]             256
             ReLU-13        [-1, 128, 112, 112]               0
        MaxPool2d-14          [-1, 128,

In [48]:
epochs = 100

max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4
opt_func = torch.optim.Adam

In [49]:
!pip install wandb

In [50]:

import wandb

name = f'cnn-{epochs}-epochs'
wandb.init(project="plantvillage-aug", name=name, config={
    "epochs": epochs,
    "learning_rate": max_lr,
    "grad_clip": grad_clip,
    "weight_decay": weight_decay,
    "opt_func": opt_func.__name__,
    "batch_size": batch_size,
    "dataset": "cnn",
    "architecture": "efficientNet"
})

'\nimport wandb\n\nname = f\'cnn-{epochs}-epochs\'\nwandb.init(project="plantvillage-aug", name=name, config={\n    "epochs": epochs,\n    "learning_rate": max_lr,\n    "grad_clip": grad_clip,\n    "weight_decay": weight_decay,\n    "opt_func": opt_func.__name__,\n    "batch_size": batch_size,\n    "dataset": "cnn",\n    "architecture": "efficientNet"\n})\n'

In [51]:
# for training
@torch.no_grad()
def evaluate(model, val_loader):
    """Evaluates the model's performance on the validation set"""
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    # return model.validation_epoch_end(outputs)
    val_losses = [x["val_loss"] for x in outputs]
    epoch_loss = torch.stack(val_losses).mean()       # Combine loss
    val_accuracies = [x["val_accuracy"] for x in outputs]
    epoch_accuracy = torch.stack(val_accuracies).mean()
    val_precisions = [x["val_precision"] for x in outputs]
    epoch_precision = torch.tensor(sum(val_precisions)/len(val_precisions))
    val_f1s = [x["val_f1"] for x in outputs]
    epoch_f1 = torch.tensor(sum(val_f1s)/len(val_f1s))
    return {"val_loss": epoch_loss.item(), "val_accuracy": epoch_accuracy.item(), "val_precision": epoch_precision.item(), "val_f1": epoch_f1.item()} # Combine accuracies


def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']


def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, weight_decay=0,
                grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []

    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # scheduler for one cycle learniing rate
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader))


    for epoch in range(epochs):
        # Training
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()

            # gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)

            optimizer.step()
            optimizer.zero_grad()

            # recording and updating learning rates
            lrs.append(get_lr(optimizer))
            sched.step()

            # logging to wandb
            wandb.log({"Train Loss": loss.item()})


        # validation
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)

        # logging to wandb
        wandb.log({
           "Epoch": epoch,
           "Val Loss": result['val_loss'],
           "Val Accuracy": result['val_accuracy'],
           "Val Precision": result['val_precision'],
           "Val F1": result['val_f1']
        })

    return history

In [52]:
%%time
history = fit_one_cycle(
  epochs,
  max_lr,
  model,
  train_loader,
  val_loader,
  grad_clip=grad_clip,
  weight_decay=1e-4,
  opt_func=opt_func
)

Epoch [0], last_lr: 0.00043, train_loss: 0.5413, val_loss: 0.4737, val_acc: 0.7500
Epoch [1], last_lr: 0.00050, train_loss: 0.3806, val_loss: 0.3745, val_acc: 0.8207
Epoch [2], last_lr: 0.00063, train_loss: 0.3522, val_loss: 0.6054, val_acc: 0.7039
Epoch [3], last_lr: 0.00081, train_loss: 0.2981, val_loss: 0.4297, val_acc: 0.8207
Epoch [4], last_lr: 0.00104, train_loss: 0.2859, val_loss: 0.5978, val_acc: 0.7714
Epoch [5], last_lr: 0.00131, train_loss: 0.2606, val_loss: 0.4622, val_acc: 0.8289
Epoch [6], last_lr: 0.00163, train_loss: 0.2739, val_loss: 0.5829, val_acc: 0.7763
Epoch [7], last_lr: 0.00198, train_loss: 0.2948, val_loss: 0.3739, val_acc: 0.8701
Epoch [8], last_lr: 0.00237, train_loss: 0.2686, val_loss: 0.7451, val_acc: 0.7829
Epoch [9], last_lr: 0.00280, train_loss: 0.2559, val_loss: 0.3722, val_acc: 0.8816
Epoch [10], last_lr: 0.00324, train_loss: 0.2077, val_loss: 0.3652, val_acc: 0.8865
Epoch [11], last_lr: 0.00371, train_loss: 0.2678, val_loss: 0.4293, val_acc: 0.8470
Ep

In [53]:
torch.save(model.state_dict(), f'models/plant_disease_aug_cnn_{epochs}.pth')

In [54]:
# clear nvidia cache
import torch, gc
gc.collect()
torch.cuda.empty_cache()